In [ ]:
import numpy as np
import pandas as pd
from glob import glob

In [ ]:
data = pd.read_csv('/resources/data/preprocessed.csv')
data


In [ ]:
import plotly.io as pio
import plotly.express as px
pio.renderers.default = "notebook"


In [ ]:
px.histogram(data, x='date', color='materia')

In [ ]:
violence_cats = [
    "violencia_de_genero",
    "v_fisica",
    "v_psic",
    "v_econ",
    "v_sex",
    "v_soc",
    "v_amb",
    "v_simb",
    "v_polit",
]
bool_violence_cats = [f"have:{cat}" for cat in violence_cats]
for cat in violence_cats:
    data[f"have:{cat}"] = data[cat].apply(lambda v: v == "si")

data["have:violence"] = data[bool_violence_cats].sum(axis=1)


In [ ]:
px.histogram(
    data,
    "have:violence",
    color="have:violencia_de_genero",
    title="nro de formas de violencia presentes en cada caso",
)


In [ ]:
cats = list(set(bool_violence_cats) - {"have:violencia_de_genero"})  #
data_ = data[cats].sum()
data_ = pd.DataFrame(data_).reset_index()
px.pie(
    data_,
    values=0,
    names="index",
    title="porcentage de formas de violencia presentes en total de datos",
)


## check

In [ ]:
# checkeo si frases se repiten en registros distintos
duplicated = (
    data.groupby(["frases_agresion"])
    .agg({"nro_registro": "nunique"})
    .sort_values(by=["nro_registro"], ascending=False)
)
duplicated


In [ ]:
caso = data.query('frases_agresion == "hija de puta me estas cagando vas a ver lo que te va a pasar no tengo nada que perder nadie te va a poder salvar"')
caso

In [ ]:

data_ = data.dropna(subset=["date"])
expedientes = data_.groupby("n_expte_eje").agg(
    {
        "nro_registro": "nunique",
        "have:violence": "sum",
    }
)
expedientes.rename(
    columns={
        "nro_registro": "nro_registros",
        "have:violence": "violence",
    },
    inplace=True,
)
expedientes["tiempo_entre_registros_maxmin"] = data_.groupby("n_expte_eje").agg(
    {"date": lambda d: pd.to_datetime(d).max() - pd.to_datetime(d).min()}
)
expedientes["tiempo_entre_registros_mean"] = data_.groupby("n_expte_eje").agg(
    {"date": lambda d: pd.to_datetime(d).diff().mean()}
)
expedientes


In [ ]:
px.histogram(
    expedientes,
    "nro_registros",
    color=expedientes["violence"].astype(bool),
    title="numero de resoluciones por caso",
)


# registros repetidos

In [ ]:
data = pd.read_csv('/resources/data/preprocessed.csv')

duplicated = data.groupby(['nro_registro', 'n_expte_eje', 'link']).nunique().sort_values(by=['n'], ascending=False)
duplicated = data.groupby(['path']).nunique().sort_values(by=['n'], ascending=False)
# data.groupby(['nro_registro', 'tomo']).nunique().sort_values(by=['n'], ascending=False)
duplicated

In [ ]:
duplicated.replace(1, np.nan).dropna(axis=1, how='all')

In [ ]:
duplicated.query('n_expte_eje > 1')

# filtros

In [ ]:
pd.set_option('display.max_colwidth', -1)
data.query('path == "/resources/data/documents/2f5a62f565b4d6940fca8acc16fcfb"')

In [ ]:
pd.set_option('display.max_colwidth', -1)
data.query('path == "/resources/data/documents/8909d6db3d91b0bc36ba258c0220e2"')

In [ ]:
DROP_REGISTRIES = [
    # n_expte_eje error
    ## nro_registro 1605; tomo 13
    1582,
    1583,
    ## nro_registro 4435; tomo 49
    5189,
    5190,
]

data.query('n not in @DROP_REGISTRIES', inplace=True)

In [ ]:
l = []
l[0] or None

In [ ]:
pd.reset_option('display.max_rows')
def get_unique_values(serie):
    values = [v for v in set(serie) if v not in [None, np.nan]]
    return values

simplified = data.groupby(['nro_registro', 'tomo', 'link']).agg(get_unique_values)

for column in simplified.columns:
    max_len = simplified[column].apply(len).max()
    if max_len <= 1:
        simplified[column] = simplified[column].apply(lambda x: x[0] if x else None)
    else: 
        print('column with multiples values:', column)

# simplified = simplified.reset_index()
simplified

In [ ]:
def _len(value):
    if isinstance(value, list):
        return len(value)
    return 1
value_sizes = simplified.applymap(_len)

In [ ]:
column = 'date'
mask = value_sizes[column] > 1
simplified.loc[mask, column].values

In [ ]:
ASSUME_UNIQUE = ['nro_registro', 'tomo', 'n_expte_eje', 'firma']

In [ ]:
# pd.set_option.display.max_rows = None
pd.set_option('display.max_rows', None)
def _len(value):
    if isinstance(value, list):
        return len(value)
    return 1
value_sizes = simplified.applymap(_len)
value_sizes_max = value_sizes.max()
value_sizes_max[value_sizes_max == 1]

In [ ]:
case = simplified.loc[1147]
case

# Casos con violencia

In [ ]:
expedientes_ = expedientes.query("nro_registros > 1 and violence > 1")
px.histogram(
    expedientes_,
    x=expedientes_["tiempo_entre_registros_maxmin"].dt.days,
    color="nro_registros",
    title="dias entre primer y ultimo registro del caso",
)


In [ ]:
expedientes_ = expedientes.query("nro_registros > 1 and violence > 1")
px.histogram(
    expedientes_,
    x=expedientes_["tiempo_entre_registros_mean"].dt.days,
    color="nro_registros",
    title="tiempo medio entre 2 registros consecutivos",
)


## modalidades

In [ ]:
set(data['modalidad_de_la_violencia'].values)

In [ ]:
registro = data.query('modalidad_de_la_violencia == "domestica"').iloc[300]
print(registro['path'])
registro